### 유사도 검색 스켈레톤 코드(최종아님)
legal-kr-sbert-contrastive 모델을 바탕으로 Bi-Encoder 방식으로 유사도 검색을 진행한 후,

그 결과에 대해 ms-marco-MiniLM 모델을 바탕으로 Cross-Encoder 방식의 검색을 수행하는 코드

In [1]:
import torch
import numpy as np

def get_device() -> torch.device:
    """
    Returns the appropriate torch device: CUDA if available, otherwise CPU.
    """
    return torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
def embed_texts(
    texts: list[str],
    model: torch.nn.Module,
    tokenizer,
    batch_size: int = 16,
    max_length: int | None = None,
    device: torch.device | None = None
) -> np.ndarray:
    """
    Embed a list of texts into vector representations using the [CLS] token embedding.

    Args:
        texts (list[str]): Input texts to embed.
        model (torch.nn.Module): Pretrained Transformer model.
        tokenizer: Corresponding HuggingFace tokenizer.
        batch_size (int): Number of samples per batch.
        max_length (int, optional): Maximum token length. Defaults to model.config.max_position_embeddings.
        device (torch.device, optional): Device for inference. Defaults to CUDA if available else CPU.

    Returns:
        np.ndarray: Array of shape (len(texts), hidden_size) with embeddings.
    """
    # Determine max_length
    if max_length is None:
        max_length = model.config.max_position_embeddings

    # Determine device
    if device is None:
        device = get_device()

    # Prepare model
    model = model.to(device)
    model.eval()

    all_embeds: list[np.ndarray] = []
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i : i + batch_size]
            encoded = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=max_length,
                return_tensors="pt"
            )
            # Move to device
            encoded = {k: v.to(device) for k, v in encoded.items()}

            outputs = model(**encoded)
            # Extract [CLS] token embedding (first token)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeds.append(cls_embeddings.cpu().numpy())

    return np.vstack(all_embeds)

In [3]:
from transformers import AutoTokenizer, AutoModel

def load_model_and_tokenizer(model_name: str):
    """
    Load a pretrained model and its tokenizer from HuggingFace.

    Args:
        model_name (str): Name or path of the model.

    Returns:
        model, tokenizer
    """
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    return model, tokenizer

c:\Users\user\Documents\JSY\KW\3-1\TextMining\TextMining\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd

df = pd.read_csv("labeled.csv")
df

,filename,label,text,basis
0,001_개인정보취급방침_가공.json,유리,제2조(개인정보의 처리 및 보유기간) \n① 협회는 법령에 따른 개인정보 보유․이용...,NaN
1,001_결혼정보서비스_가공.json,유리,제3조 (회원가입)\n① 회원이 되려고 하는 사람은 결혼관련 개인정보를 회사에 제공...,NaN
2,001_보증_가공.json,유리,제2조(보증금액)\n ① 이 보증서에 의한 보증금액은 채권자의 채무자에 대한 보증부...,NaN
3,001_사이버몰_가공.json,유리,제3조 (약관 등의 명시와 설명 및 개정)\n① 몰은 이 약관의 내용과 상호 및 대...,NaN
4,001_상해보험_가공.json,유리,제4조(보험금 지급에 관한 세부규정)\n② 제3조(보험금의 지급사유) 제2호에서 장...,NaN
...,...,...,...,...
7995,620_임대차계약_가공.json,불리,제2조(임대료)\n제2항 임대료 등의 연체시 매월 100분의 10에 해당하는 연체료...,매월 100분의 10은 연120퍼센트의 연체료율이 되어 이자제한법상 연25퍼센트를 ...
7996,621_임대차계약_가공.json,불리,제12조(해약인정 및 보증금 등의 반환)\n제2항 갑은 해약이 확정되면 즉시 을의 ...,점포를 재임대하여 반환하거나 또는 해약일로부터 3개월 이내에 임대보증금을 반환하되 ...
7997,622_임대차계약_가공.json,불리,제17조(임대인의 금지사항)\n제2항 을이 전항 및 제11조와 제14조에 위반 내지...,임대차계약관계에서 임차인이 지켜야 할 사항을 게을리하거나 지키지 않는 경우 임대인은...
7998,623_임대차계약_가공.json,불리,제20조(임대인의 금지조항)\n제1항 본 계약 각 조항의 해석상의 이의가 있는 경우...,해석상 이의가 있는 경우 임대인의 해석에 따르고 계약에 명시되지 않은 사항은 임대인...


In [1]:
from transformers import BertModel, BertTokenizer
from sklearn.neighbors import NearestNeighbors
import torch

c:\Users\user\Documents\JSY\KW\3-1\TextMining\TextMining\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from sentence_transformers import SentenceTransformer

# 1) 모델 디렉터리 경로 지정 (훈련 시 output_dir 과 동일)
model_dir = os.path.abspath("legal-kr-sbert-contrastive")

# 2) 경로 확인
if not os.path.isdir(model_dir):
    raise FileNotFoundError(f"모델 디렉터리({model_dir})가 존재하지 않습니다.")

# 3) SBERT 포맷 모델 로드 (로컬만 사용)
model = SentenceTransformer(
    model_name_or_path=model_dir,
    local_files_only=True,
    device="cpu"
)

In [5]:
texts = df["text"].fillna("").astype(str).tolist()
embeddings = model.encode(
    texts,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    truncation=True,        # ensure longer texts are truncated
    max_length=128          # or whatever max length you used in training
)

Batches: 100%|██████████| 500/500 [02:17<00:00,  3.64it/s]


In [6]:
nn = NearestNeighbors(n_neighbors=5, metric="cosine")
nn.fit(embeddings)

NearestNeighbors(metric='cosine')

In [7]:
from sklearn.neighbors import NearestNeighbors

# 이미 embeddings (NumPy array, shape: [n_samples, dim])를 계산해 두고,
# nn = NearestNeighbors(n_neighbors=5, metric='cosine').fit(embeddings)

def find_similar(query: str, top_k: int = 5):
    # 1) 쿼리 임베딩: SBERT encode() 사용
    query_emb = model.encode(
        [query],
        batch_size=1,
        convert_to_numpy=True,
        show_progress_bar=False,
        truncation=True,
        max_length=128  # 학습 때 사용한 max_length와 동일하게
    )
    # 2) 최근접 이웃 검색
    distances, indices = nn.kneighbors(query_emb, n_neighbors=top_k)
    # 3) 거리 → 유사도로 변환 후 결과 조합
    results = []
    for dist, idx in zip(distances[0], indices[0]):
        sim = 1.0 - dist  # cosine 거리 → 유사도
        filename = df.loc[idx, "filename"]
        text     = df.loc[idx, "text"]
        label     = df.loc[idx, "label"]
        results.append((filename, text, label, sim))
    return results


In [9]:
if __name__ == "__main__":
    query = "임대인은 임차인의 동의 없이 언제든지 본 계약상의 임대료 및 관리비 금액을 개정할 수 있다."
    results = find_similar(query, top_k=15)

In [11]:
import pandas as pd

# 1) 함수 호출
results = find_similar(query, top_k=15)

# 2) 리스트를 DataFrame으로 변환
df_results = pd.DataFrame(
    results,
    columns=["filename", "text", "label", "similarity"]
)

# 3) 확인
df_results

,filename,text,label,similarity
0,205_임대차계약_가공.json,제4조\n임차인은 임대료 및 각종 공과금은 매월 말일 임대인의 주소지에서 후불로 지...,불리,0.983281
1,275_임대차계약_가공.json,제17조(특약)\n제4항 임차인의 의무\n제2호 임차인은 추가로 설치하는 시설물에...,불리,0.983137
2,372_임대차계약_가공.json,제14조(명도와 원상복구)\n제4항 임차인은 임대인 및 시행위탁사에게 권리금 및 유...,불리,0.981545
3,207_임대차계약_가공.json,제9조\n임차인은 건물의 부분적인 통상 범위의 수선은 임차인의 비용으로 행하고 임대...,불리,0.980633
4,429_임대차계약_가공.json,제1조(전세금의 변제충당 불가)\n 임대인 은 어떠한 경우라도 본 계약에 의한 임대...,불리,0.979767
5,232_임대차계약_가공.json,제3조(임대보증금)\n제3항 이 계약이 제3조제2항에 따라 자동적으로 해지되었을 때...,불리,0.979089
6,139_임대차계약_가공.json,"제16조(임대인의 면책사항)\n제1항 임대인은 천재지변, 지진, 풍수해, 전쟁, 폭...",불리,0.979005
7,208_임대차계약_가공.json,제3조 \n임대료금은 하기와 같다.\n보 증 금 : 일금 정(￦ ...,불리,0.978887
8,209_임대차계약_가공.json,제12조(계약의 해제 또는 해지)\n제1항 계약일반조건 제10조 각 호의 사유 의외...,불리,0.978718
9,289_임대차계약_가공.json,제16조(임대차 등기 등)\n제2항 임차인은 관계법령의 규정에 따라 임대인의 사전동...,불리,0.978399


교차인코딩 적용

In [12]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder(
    "cross-encoder/ms-marco-MiniLM-L-6-v2",
    device="cpu"
)


In [13]:
def rerank_with_cross_encoder(query: str,
                              top_k_bi: int = 15,
                              top_k_final: int = 5):
    """
    1) bi-encoder로 top_k_bi 후보 추출 (filename, text, label, bi_score)
    2) cross-encoder로 재점수화
    3) cross_score 기준 내림차순으로 top_k_final 반환
    """
    # 1) bi-encoder 기반 후보 추출
    #    find_similar()가 (filename, text, label, bi_score) 튜플을 반환한다고 가정
    candidates = find_similar(query, top_k=top_k_bi)
    filenames, texts, labels, bi_scores = zip(*candidates)

    # 2) CrossEncoder 입력 생성
    pairs = [(query, txt) for txt in texts]
    cross_scores = cross_encoder.predict(pairs)

    # 3) bi_score, cross_score, label 함께 묶어 재정렬
    reranked = sorted(
        zip(filenames, texts, labels, bi_scores, cross_scores),
        key=lambda x: x[4],  # cross_score 기준
        reverse=True
    )

    # 4) 최종 top_k_final 반환
    return [
        {
            "filename":    fn,
            "text":        txt,
            "label":       lbl,
            "bi_score":    float(bs),
            "cross_score": float(cs)
        }
        for fn, txt, lbl, bs, cs in reranked[:top_k_final]
    ]


In [20]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# 테스트

In [21]:
if __name__ == "__main__":
    query = "가맹점주는 본 계약의 유효기간 및 계약 종료 후 1년간 본사 또는 본사가 지정하는 제3자로부터만 가맹점 운영에 필요한 모든 원‧부자재, 소모품, 홍보물 등을 구매해야 하며, 이를 위반할 경우 본사는 별도의 통지 없이 즉시 계약을 해지할 수 있다. 이 경우 가맹점주는 위약금으로 남은 계약기간 동안 본사가 주장하는 예상 로열티 총액의 100%에 상당하는 금액을 본사에 지급하여야 한다."
    results = rerank_with_cross_encoder(query, top_k_bi=15, top_k_final=3)
    
    # 리스트 → DataFrame 변환
    df_results = pd.DataFrame(results)
    
    # 소수점 네 자리까지 포맷팅
    pd.options.display.float_format = "{:.4f}".format
    

In [22]:
df_results

,filename,text,label,bi_score,cross_score
0,101_가맹계약_가공.json,"제4조(가맹비)\n(을)은 가맹계약시 가맹비 500만원을 (갑)에게 지급하여야 한다.\n제1항 가맹비는 점포개설에 따른 주방조리교육, 마케팅교육, 브랜드 사용료, 운영매뉴얼 제공비, 지속적인 가맹점 관리와 브랜드 광고 홍보비를 포함한다.\n제1항 가맹비는(을)의 가맹점 영업 개시와 함께 소모되는 것으로서 (을)은(갑)에게 이에 대한 반환을 청구할 수 없다.",불리,0.9834,3.8624
1,273_가맹계약_가공.json,"제26조 \n집행비용 가맹본부가 본 계약상의 권리를 확보 및 보호하거나, 본 계약의 조건을 집행하기 위해 가맹점사업자에 대해 소송을 제기하는 경우, 가맹본부는 승소판정에 의하여 인정된 권리 이외에 변호사 비용을 포함하여 법원이 합당하다고 인정하는 범위의 소송비용을 회수할 권리를 갖는다.",불리,0.9847,3.6348
2,386_가맹계약_가공.json,제14조 (가맹비) \n3항 본 계약 기간 내에 계약이 해지된 경우 갑 은 을 로부터 지급받은 가맹비에 대하여 각호 1의 규정에 따라 반환한다. 제14조 \n3항 1호 가맹사업 개시 이전에 계약이 해지된 경우 갑 은 을 에게 인도된 제품 및 제공된 서비스(노하우 및 정보제공의 대가 등)의 비용을 제외한 금액을 반환한다. 제14조 \n3항2호 가맹사업 개시 후 계약이 해지된 경우 가맹비는 을 이 000 가맹사업에 가맹점으로 참여하기 위하여 최초 발생하는 소멸성 비용으로서 가맹사업 개시 후에는 반환되지 않는다.\n,불리,0.9836,3.5962


In [23]:
if __name__ == "__main__":
    query = "임대인 및 임차인은 본 계약에 관하여 발생하는 일체의 소송, 중재, 분쟁해결 절차와 관련하여 임대인의 본사 소재지를 관할하는 대한민국 법원을 단독·배타적 제1심 관할법원으로 하는 데 합의한다."
    results = rerank_with_cross_encoder(query, top_k_bi=15, top_k_final=3)
    
    # 리스트 → DataFrame 변환
    df_results = pd.DataFrame(results)
    
    # 소수점 네 자리까지 포맷팅
    pd.options.display.float_format = "{:.4f}".format

In [24]:
df_results

,filename,text,label,bi_score,cross_score
0,148_임대차계약_가공.json,"제14조(유보금 및 계약일반조건 제8조의 보충)\n제2항 계약일반조건 제8조 제2항에도 불구하고, 임대인이 주택 및 내부일체에 대한 점검시 권한있는 제3자에 의한 검증이 필요하다고 판단할 때에는 임차인에게 반환해야 할 보증금 중 유보금 오백만원을 우선 공제한 후 임대인이 선정한 권한있는 제3자의 검증을 거친다.\n제3항 임대인은 공제한 유보금 중 다음 각 호에 해당하는 금액을 공제한 후 유보금 공제일로부터 3개월 이내에 환불한다. 제15조(원상회복)\n제2항 퇴거를 위한 시설점검 후 원상복구가 완료되지 않았거나 원상복구 비용의 예측이 불가한 경우 계약특수조건 제14조에서와 같이 보증금 일부(5백만원)를 우선 공제한 후 검증결과에 따라 정산한다.",불리,0.9747,3.7048
1,222_임대차계약_가공.json,"제15조(계약일반조건 8조의 보충)\n계약일반조건 제8조 제2항에도 불구하고, 임대인이 주택 및 내부일체에 대한 점검시 권한있는 제3자에 의한 검증이 필요하다고 판단할 때에는 임차인에게 반환해야 할 보증금 중 유보금 오백만원을 우선 공제한 후 임대인과 임차인이 협의하여 선정한 권한있는 제3자의 검증을 거친다. 제16조(원상회복)\n제2항 퇴거를 위한 시설점검 후 원상복구가 완료되지 않았거나 비용 예측이 불가한 경우 계약특수조건 제15조에서와 같이 보증금 일부(5백만원)를 우선 공제한 후 검증결과에 따라 정산한다.",불리,0.9747,3.0210
2,134_임대차계약_가공.json,"제5조(임대차조건의 조정)\n제1항 임대인은 임대차목적물에 관한 조세, 공과금, 물가상승율, 그 밖의 부담의 증감이나 경제사정의 변동 등의 제반 사정을 고려하여 매년 월 최소보장 임대료 및 월 관리비를 조정한다.\n제2항 임대인은 제1항에 따른 연 단위의 임대료 및 관리비의 조정 이외에도 임대차목적물 주변 환경의 현격한 변화, 조세, 공과금, 물가의 급격한 상승, 기타 급격한 경제 여건의 변동 등 부득이한 사유가 발생할 경우 그 사유를 임차인에게 통지하고, 위와 같은 변경된 사정을 반영하여 임대 형식, 임대보증금 등 임대차조건을 변경할 수 있다.\n",불리,0.9754,2.8163


# Generation 연결

In [25]:
from typing import List
from langchain.schema import Document, BaseRetriever

class CustomRetriever(BaseRetriever):
    # Pydantic field 로 인식되도록 어노테이션 처리
    top_k: int = 5

    def get_relevant_documents(self, query: str) -> List[Document]:
        # rerank_with_cross_encoder 는 (filename, text, label, bi_score, cross_score) 반환
        hits = rerank_with_cross_encoder(query, top_k_bi=15, top_k_final=self.top_k)
        docs = []
        for hit in hits:
            docs.append(
                Document(
                    page_content=hit["text"],
                    metadata={
                        "filename":    hit["filename"],
                        "label":       hit["label"],
                        "bi_score":    hit["bi_score"],
                        "cross_score": hit["cross_score"],
                    }
                )
            )
        return docs

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        # 비동기 호출이 필요할 때
        return self.get_relevant_documents(query)


C:\Users\user\AppData\Local\Temp\ipykernel_24512\392527269.py:4: DeprecationWarning: Retrievers must implement abstract `_get_relevant_documents` method instead of `get_relevant_documents`
  class CustomRetriever(BaseRetriever):
C:\Users\user\AppData\Local\Temp\ipykernel_24512\392527269.py:4: DeprecationWarning: Retrievers must implement abstract `_aget_relevant_documents` method instead of `aget_relevant_documents`
  class CustomRetriever(BaseRetriever):


In [26]:
retriever = CustomRetriever(top_k=5)


In [28]:
from langchain.schema import Document, BaseRetriever
from langchain.llms import Ollama
from langchain.chains import RetrievalQA

# 3) Ollama LLM 인스턴스화
llm = Ollama(
    model="llama2",
    base_url="http://127.0.0.1:11434"
)

# 4) RetrievalQA 체인 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True
)

# 5) 질의 실행 및 출력
input_clause = (
    "가맹점주는 본 계약의 유효기간 및 계약 종료 후 1년간 본사 또는 "
    "본사가 지정하는 제3자로부터만 가맹점 운영에 필요한 모든 원·부자재를 "
    "구매해야 하며, 이를 위반 시 즉시 계약 해지 및 과도한 위약금이 부과된다."
)

prompt = f"""
다음 조항과 유사한 기존 조항 5개의 “유불리(label)”를 근거로,
주어진 조항의 유불리 여부를 판단하고 그 이유를 간단히 서술하라.

---
### 주어진 조항
“{input_clause}”
"""

result = qa_chain(prompt)

print("🔍 Retrieved passages:")
for doc in result["source_documents"]:
    label = doc.metadata.get("label", "N/A")
    snippet = doc.page_content.replace("\n", " ")[:100]
    print(f"- [label: {label}] {snippet}...")

print("\n💡 판단 결과:")
print(result["result"])


C:\Users\user\AppData\Local\Temp\ipykernel_24512\1498066287.py:35: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain(prompt)


🔍 Retrieved passages:
- [label: 불리] 제14조 (가맹비)  3항 본 계약 기간 내에 계약이 해지된 경우  갑 은  을 로부터 지급받은 가맹비에 대하여 각호 1의 규정에 따라 반환한다. 제14조  3항 1호 가맹사업 개...
- [label: 불리] 제26조  집행비용 가맹본부가 본 계약상의 권리를 확보 및 보호하거나, 본 계약의 조건을 집행하기 위해 가맹점사업자에 대해 소송을 제기하는 경우, 가맹본부는 승소판정에 의하여 인정...
- [label: 불리] 제21조(가맹점의 의무 및 금지행위)  제6항 을은 갑의 가맹점 기본 운영방침에 위배되는 행위를 할 수 없고, 갑이 공급하는 상품 이외의 상품을 판매할 수 없다.  "  제25조(...
- [label: 불리] 제23조제8항(기타규정) 가맹계약자는 본 계약과 관련하여 가맹사업자가 부담한 모든 변호사 비용(본 계약의 법적인 이행과 관련하여 부담한 인지대 및 비용을 포함하나 이에 한정하지 않...
- [label: 불리] (특약사항) 제10조 제1항 가맹계약*자는 가맹사업*자가 허가한 영업용 전화번호 (        )를 사용하여야 한다. 단, 가맹점에 추가로 영업용 전화번호 개설을 원할 시는 가맹...

💡 판단 결과:
Thank you for providing the context. Based on the given contract, I can give my analysis of each label and whether they are likely to be considered as breaches or not.

1. 유불리(label) - "must purchase"

* Reason: The contract specifies that the mall owner must purchase all necessary materials for mall operation from the designated third party within a year after the contract's